# Synthea Medgraph

***Synthetic Patient Dataset Generation***

Open-source healthcare data generator that models the medical history of synthetic patients. The resulting dataset is free of PHI or PHI.

## Install pyTigerGraph

In [1]:
from IPython.display import clear_output
!pip install -qq watermark

################################
# Packages to install
################################
!pip install -U -qq pyTigerGraph

################################

clear_output()
%reload_ext watermark
%watermark -v -p numpy,pyTigerGraph

Python implementation: CPython
Python version       : 3.9.10
IPython version      : 8.0.1

numpy       : 1.22.2
pyTigerGraph: 0.0.9.9.2



***Imports***

In [2]:
import pyTigerGraph as tg
import json
import pandas as pd

***Helper Functions***

In [3]:
def pprint(input):
  print(json.dumps(input, indent=2))

***Connection parameters***

In [4]:
hostName = "http://localhost"
userName = "tigergraph"
password = "tigergraph"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

## Understanding the Schema

![](https://i.ibb.co/CQXYbH9/Screenshot-2021-08-24-003804.png)

At it's core, this schema repesents all of a patient's interactions with the healthcare system. It also includes demographic information about each patient including their age, race, gender, and location. The full schema can be found on the [Synthea Data Directory](https://github.com/synthetichealth/synthea/wiki/CSV-File-Data-Dictionary).

Interactions with the healthace system are stored as `Encounters` and those `Encounters` nodes will connect to nodes expressing the nature of each `Encounter` such as a `Procedure` with a physician or a `Medication` perscription.

**A Critical part of this graph is the `SnomedCode` vertex.** This vertex type is tied to almost every type of `Encounters` and contains the Snomed code that describes each `Encounters`. For those unfamilar with Snomed codes, they are standardized codes used to describe the nature of a medical encounter. For example:
* Snomed code **31978002** describes a **fracture of the tibia**
* Snomed code **86174004** signifes the use of a **laparoscope** medical device
* Snomed code **25876001** denotes that a procedure is of **emergency** priority

You can read more about Snomed [here](https://confluence.ihtsdotools.org/display/DOCSTART/4.+SNOMED+CT+Basics)

Below is an overview of some key aspects of the schema and a look at their attributes. Where `_code` is used, such as `allergy_code` that is listing a **Snomed code**. These codes are also expressed as edges that lead to Nodes representing each **Snomed code**.


### <center>Encounter Related Vertices

#### <center>Encounter

|Attribute Name|Attribute Type|
|---|---|
|PRIMARY_ID encounter_id|STRING|
|baseEncounterCost|DOUBLE|
|totalClaimCost|DOUBLE|
|payerCoverage|DOUBLE|
|classType|STRING|
|startTime|DATETIME|
|endTime|DATETIME|

#### <center>Notes

|Attribute Name|Attribute Type|
|---|---|
|PRIMARY_ID note_id|STRING|
|chiefComplaint|STRING|
|historyOfPresentIllness|STRING|
|socialHistory|STRING|
|allergies|STRING|
|medications|STRING|
|assessment|STRING|
|plan|STRING|

#### <center>Symptoms

|Attribute Name|Attribute Type|
|---|---|
|PRIMARY_ID symptom_id|STRING|
|symptom|STRING|
|symptomValue|INT|
|pathology|STRING|

#### <center>Medication

|Attribute Name|Attribute Type|
|---|---|
|PRIMARY_ID medication_id |STRING|
|medication_code|STRING|
|description|STRING|
|startDate|DATETIME|
|endDate|DATETIME|
|baseCost|DOUBLE|
|payerCoverage|DOUBLE|
|dispenses|INT|
|totalCost|DOUBLE|

#### <center>Procedures

|Attribute Name|Attribute Type|
|---|---|
|PRIMARY_ID procedure_id|STRING|
|procedure_code|STRING|
|description|STRING|
|baseCost|INT|
|dateOfProcedure|DATETIME|

#### <center>Observations

|Attribute Name|Attribute Type|
|---|---|
|PRIMARY_ID observation_id|STRING|
|dateOfObservation|DATETIME|
|observation_code|STRING|
|description|STRING|
|obsValue|STRING|
|units|STRING|

#### <center>Imaging Studies

|Attribute Name|Attribute Type|
|---|---|
|PRIMARY_ID imaging_id|STRING|
|bodySiteCode|STRING|
|bodySiteDescription|STRING|
|modalityCode|STRING|
|modalityDescription|STRING|
|SOPCode|STRING|
|SOPDescription|STRING|
|dateOfImage|DATETIME|

## Loading the Schema

We'll be creating the schema through a series of **GSQL** queries that we'll execute through pyTigerGraph.

#### `Patient` related Vertices

In [5]:
conn.gsql(
    """
CREATE VERTEX Gender(PRIMARY_ID gender_id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Race(PRIMARY_ID race_id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Ethnicity(PRIMARY_ID ethnicity_id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Patient(PRIMARY_ID patient_id STRING, lastName STRING, firstName STRING, maiden STRING,
                      birthday DATETIME, ssn STRING, license STRING, passport STRING,
                      healthcareExpense DOUBLE, healthcareCoverage DOUBLE, suffix STRING, prefix STRING, 
                      maritalStatus STRING, birthplace STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Allergies(PRIMARY_ID allergy_id STRING, allergy_code STRING, description STRING, 
                        startDate DATETIME, endDate DATETIME) WITH primary_id_as_attribute="true"
CREATE VERTEX Immunizations(PRIMARY_ID immunization_id STRING, immunization_code STRING, description STRING, 
                            dateOfImmunization DATETIME, baseCost DOUBLE) WITH primary_id_as_attribute="true"
"""
)

'Successfully created vertex types: [Gender].\nSuccessfully created vertex types: [Race].\nSuccessfully created vertex types: [Ethnicity].\nSuccessfully created vertex types: [Patient].\nSuccessfully created vertex types: [Allergies].\nSuccessfully created vertex types: [Immunizations].'

#### `Location` related Vertices

In [6]:
conn.gsql(
    """
CREATE VERTEX AddressSynthea(PRIMARY_ID address_id STRING, name STRING, lat DOUBLE, lon DOUBLE) WITH primary_id_as_attribute="true"
CREATE VERTEX CitySynthea(PRIMARY_ID city_id STRING, cityName STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX StateSynthea(PRIMARY_ID state_id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX CountySynthea(PRIMARY_ID county_id STRING, countyName STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX ZipCodeSynthea(PRIMARY_ID zip_id STRING) WITH primary_id_as_attribute="true"
"""
)

'Successfully created vertex types: [AddressSynthea].\nSuccessfully created vertex types: [CitySynthea].\nSuccessfully created vertex types: [StateSynthea].\nSuccessfully created vertex types: [CountySynthea].\nSuccessfully created vertex types: [ZipCodeSynthea].'

#### `Encounter` related Vertices

In [7]:
conn.gsql(
    """
CREATE VERTEX Encounter(PRIMARY_ID encounter_id STRING, baseEncounterCost DOUBLE, totalClaimCost DOUBLE,
                        payerCoverage DOUBLE, classType STRING, startTime DATETIME, endTime DATETIMe) WITH primary_id_as_attribute="true"
CREATE VERTEX Notes(PRIMARY_ID note_id STRING, chiefComplaint STRING, historyOfPresentIllness STRING,
                        socialHistory STRING, allergies STRING, medications STRING, assessment STRING, plan STRING) with primary_id_as_attribute="true"
CREATE VERTEX Symptoms(PRIMARY_ID symptom_id STRING, symptom STRING, symptomValue DOUBLE, pathology STRING) with primary_id_as_attribute="true"
CREATE VERTEX Medication(PRIMARY_ID medication_id STRING, medication_code STRING, description STRING,
                        startDate DATETIME, endDate DATETIME, baseCost DOUBLE, payerCoverage DOUBLE, dispenses INT, totalCost DOUBLE) WITH primary_id_as_attribute="true"
CREATE VERTEX Procedures(PRIMARY_ID procedure_id STRING, procedure_code STRING, description STRING, baseCost DOUBLE, dateOfProcedure DaTETIME) WITH primary_id_as_attribute="true"
CREATE VERTEX Observations(PRIMARY_ID observation_id STRING, dateOfObservation DATETIME, observation_code STRING,
                        description STRING, obsValue STRING, units STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX ImagingStudies(PRIMARY_ID imaging_id STRING, bodySiteCode STRING, bodySiteDescription STRING,
                        modalityCode STRING, modalityDescription STRING, SOPCode STRING, SOPDescription STRING, dateOfImage DATETIME) WITH primary_id_as_attribute="true"
CREATE VERTEX SnomedCode(PRIMARY_ID snomed_code STRING, description STRING) WITH primary_id_as_attribute="true"
"""
)

'Successfully created vertex types: [Encounter].\nSuccessfully created vertex types: [Notes].\nSuccessfully created vertex types: [Symptoms].\nSuccessfully created vertex types: [Medication].\nSuccessfully created vertex types: [Procedures].\nSuccessfully created vertex types: [Observations].\nSuccessfully created vertex types: [ImagingStudies].\nSuccessfully created vertex types: [SnomedCode].'

#### Provider and Other Vertices

In [8]:
conn.gsql(
    """
CREATE VERTEX Conditions(PRIMARY_ID condition_id STRING, condition_code STRING, description STRING, startDate DATETIME, endDate DATETIME) WITH primary_id_as_attribute="true"
CREATE VERTEX Organizations(PRIMARY_ID organization_id STRING, name STRING, revenue DOUBLE, utilization INT, phone STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Providers(PRIMARY_ID provider_id STRING, name STRING, utilization INT, speciality STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Attribute(PRIMARY_ID label STRING, attributeValue INT) WITH primary_id_as_attribute="true"
CREATE VERTEX Device(PRIMARY_ID UDI_code STRING, description STRING, startDate DATETIME, endDate DATETIME) WITH primary_id_as_attribute="true"
CREATE VERTEX Careplans(PRIMARY_ID careplan_id STRING, description STRING, startDate DATETIME, endDate DATETIME) WITH primary_id_as_attribute="true"
CREATE VERTEX Payer(PRIMARY_ID payer_id STRING, name STRING, phone STRING, amountCovered DOUBLE, amountUncovered DOUBLE, revenue DOUBLE,
                    coveredEncounters INT, uncoveredEncounters INT, coveredMedications INT, uncoveredMedications INT,
                    coveredProcedures INT, uncoveredProcedures INT, coveredImmunizations INT, uncoveredImmunizations INT,
                    uniqueCustomers INT, QOLS_Avg DOUBLE, memberMonths INT) WITH primary_id_as_attribute="true"
"""
)

'Successfully created vertex types: [Conditions].\nSuccessfully created vertex types: [Organizations].\nSuccessfully created vertex types: [Providers].\nSuccessfully created vertex types: [Attribute].\nSuccessfully created vertex types: [Device].\nSuccessfully created vertex types: [Careplans].\nSuccessfully created vertex types: [Payer].'

### Create Edges

#### `Patient` related Edges

In [9]:
conn.gsql(
    """
CREATE UNDIRECTED EDGE PATIENT_HAS_ATTRIBUTE(FROM Patient, TO Attribute)
CREATE UNDIRECTED EDGE PATIENT_HAS_SYMPTOM(FROM Patient, TO Symptoms, ageBegin INT, ageEnd INT)
CREATE UNDIRECTED EDGE PATIENT_NOTE(FROM Patient, TO Notes, dateOfNote DATETIME)
CREATE UNDIRECTED EDGE PATIENT_GENDER(FROM Patient, TO Gender)
CREATE UNDIRECTED EDGE PATIENT_ADDRESS(FROM Patient, TO AddressSynthea)
CREATE UNDIRECTED EDGE PATIENT_RACE(FROM Patient, TO Race)
CREATE UNDIRECTED EDGE PATIENT_ETHNICITY(FROM Patient, TO Ethnicity)
CREATE UNDIRECTED EDGE PATIENT_HAS_ALLERGY(FROM Allergies, TO Patient)
CREATE UNDIRECTED EDGE PATIENT_HAS_DEVICE(FROM Device, TO Patient)
CREATE UNDIRECTED EDGE PATIENT_HAS_MEDICATION(FROM Medication, TO Patient)
CREATE UNDIRECTED EDGE PATIENT_HAS_CAREPLAN(FROM Careplans, TO Patient)
CREATE UNDIRECTED EDGE PATIENT_HAS_CONDITION(FROM Conditions, TO Patient)
CREATE UNDIRECTED EDGE PATIENT_HAS_IMMUNIZATION(FROM Immunizations, TO Patient)
CREATE UNDIRECTED EDGE PATIENT_HAS_IMAGING(FROM ImagingStudies, TO Patient)
CREATE UNDIRECTED EDGE PATIENT_HAS_PROCEDURE(FROM Procedures, TO Patient)
"""
)

'Successfully created edge types: [PATIENT_HAS_ATTRIBUTE].\nSuccessfully created edge types: [PATIENT_HAS_SYMPTOM].\nSuccessfully created edge types: [PATIENT_NOTE].\nSuccessfully created edge types: [PATIENT_GENDER].\nSuccessfully created edge types: [PATIENT_ADDRESS].\nSuccessfully created edge types: [PATIENT_RACE].\nSuccessfully created edge types: [PATIENT_ETHNICITY].\nSuccessfully created edge types: [PATIENT_HAS_ALLERGY].\nSuccessfully created edge types: [PATIENT_HAS_DEVICE].\nSuccessfully created edge types: [PATIENT_HAS_MEDICATION].\nSuccessfully created edge types: [PATIENT_HAS_CAREPLAN].\nSuccessfully created edge types: [PATIENT_HAS_CONDITION].\nSuccessfully created edge types: [PATIENT_HAS_IMMUNIZATION].\nSuccessfully created edge types: [PATIENT_HAS_IMAGING].\nSuccessfully created edge types: [PATIENT_HAS_PROCEDURE].'

#### `Medication` Edges

In [10]:
conn.gsql(
    """
CREATE UNDIRECTED EDGE MEDICATION_PAYER(FROM Medication, TO Payer)
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_MEDICATION(FROM Medication, TO Encounter)
CREATE UNDIRECTED EDGE MEDICATION_REASON_CODE(FROM Medication, TO SnomedCode)
CREATE UNDIRECTED EDGE MEDICATION_CODE(FROM Medication, TO SnomedCode)
"""
)

'Successfully created edge types: [MEDICATION_PAYER].\nSuccessfully created edge types: [ENCOUNTER_FOR_MEDICATION].\nSuccessfully created edge types: [MEDICATION_REASON_CODE].\nSuccessfully created edge types: [MEDICATION_CODE].'

#### `Location` Edges

In [11]:
conn.gsql(
    """
CREATE UNDIRECTED EDGE ADDRESS_CITY_SYNTHEA(FROM AddressSynthea, TO CitySynthea)
CREATE UNDIRECTED EDGE ADDRESS_COUNTY_SYNTHEA(FROM AddressSynthea, TO CountySynthea)
CREATE UNDIRECTED EDGE ADDRESS_ZIPCODE_SYNTHEA(FROM AddressSynthea, TO ZipCodeSynthea)
CREATE UNDIRECTED EDGE STATE_HAS_COUNTY_SYNTHEA(FROM StateSynthea, TO CountySynthea)
CREATE UNDIRECTED EDGE COUNTY_HAS_CITY_SYNTHEA(FROM CountySynthea, TO CitySynthea)
CREATE UNDIRECTED EDGE CITY_HAS_ZIPCODE_SYNTHEA(FROM CitySynthea, TO ZipCodeSynthea)
"""
)

'Successfully created edge types: [ADDRESS_CITY_SYNTHEA].\nSuccessfully created edge types: [ADDRESS_COUNTY_SYNTHEA].\nSuccessfully created edge types: [ADDRESS_ZIPCODE_SYNTHEA].\nSuccessfully created edge types: [STATE_HAS_COUNTY_SYNTHEA].\nSuccessfully created edge types: [COUNTY_HAS_CITY_SYNTHEA].\nSuccessfully created edge types: [CITY_HAS_ZIPCODE_SYNTHEA].'

#### `Encounter` Edges

In [12]:
conn.gsql(
    """
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_ALLERGY(FROM Allergies, TO Encounter)
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_DEVICE(FROM Device, TO Encounter)
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_PROCEDURE(FROM Procedures, TO Encounter)
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_CAREPLAN(FROM Careplans, TO Encounter)
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_CONDITION(FROM Conditions, TO Encounter)
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_IMMUNIZATION(FROM Immunizations, TO Encounter)
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_OBSERVATION(FROM Observations, TO Encounter)
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_IMAGING(FROM ImagingStudies, TO Encounter)
CREATE UNDIRECTED EDGE ENCOUNTER_FOR_PATIENT(FROM Encounter, TO Patient)
CREATE UNDIRECTED EDGE ENCOUNTER_UNDER_ORGANIZATION(FROM Encounter, TO Organizations)
CREATE UNDIRECTED EDGE ENCOUNTER_HAS_PROVIDER(FROM Encounter, TO Providers)
CREATE UNDIRECTED EDGE ENCOUNTER_HAS_PAYER(FROM Encounter, TO Payer)
CREATE UNDIRECTED EDGE ENCOUNTER_CODE(FROM Encounter, TO SnomedCode)
CREATE UNDIRECTED EDGE ENCOUNTER_REASON_CODE(FROM Encounter, TO SnomedCode)
"""
)

'Successfully created edge types: [ENCOUNTER_FOR_ALLERGY].\nSuccessfully created edge types: [ENCOUNTER_FOR_DEVICE].\nSuccessfully created edge types: [ENCOUNTER_FOR_PROCEDURE].\nSuccessfully created edge types: [ENCOUNTER_FOR_CAREPLAN].\nSuccessfully created edge types: [ENCOUNTER_FOR_CONDITION].\nSuccessfully created edge types: [ENCOUNTER_FOR_IMMUNIZATION].\nSuccessfully created edge types: [ENCOUNTER_FOR_OBSERVATION].\nSuccessfully created edge types: [ENCOUNTER_FOR_IMAGING].\nSuccessfully created edge types: [ENCOUNTER_FOR_PATIENT].\nSuccessfully created edge types: [ENCOUNTER_UNDER_ORGANIZATION].\nSuccessfully created edge types: [ENCOUNTER_HAS_PROVIDER].\nSuccessfully created edge types: [ENCOUNTER_HAS_PAYER].\nSuccessfully created edge types: [ENCOUNTER_CODE].\nSuccessfully created edge types: [ENCOUNTER_REASON_CODE].'

#### `Provider` and `Payer` Edges

In [13]:
conn.gsql(
    """
CREATE UNDIRECTED EDGE PROVIDER_HAS_ORGANIZATION(FROM Providers, TO Organizations)
CREATE UNDIRECTED EDGE PROVIDER_GENDER(FROM Providers, TO Gender)
CREATE UNDIRECTED EDGE PROVIDER_ADDRESS(FROM Providers, TO AddressSynthea)
CREATE UNDIRECTED EDGE PAYER_TRANSITION(FROM Payer, TO Patient, startYear DATETIME, endYear DATETIME, ownership STRING)
CREATE UNDIRECTED EDGE PAYER_ADDRESS(FROM Payer, TO AddressSynthea)
"""
)

'Successfully created edge types: [PROVIDER_HAS_ORGANIZATION].\nSuccessfully created edge types: [PROVIDER_GENDER].\nSuccessfully created edge types: [PROVIDER_ADDRESS].\nSuccessfully created edge types: [PAYER_TRANSITION].\nSuccessfully created edge types: [PAYER_ADDRESS].'

#### Additional Edges

In [14]:
conn.gsql(
    """
CREATE UNDIRECTED EDGE ALLERGY_CODE(FROM Allergies, TO SnomedCode)
CREATE UNDIRECTED EDGE DEVICE_CODE(FROM Device, TO SnomedCode)
CREATE UNDIRECTED EDGE PROCEDURE_CODE(FROM Procedures, TO SnomedCode)
CREATE UNDIRECTED EDGE PROCEDURE_REASON_CODE(FROM Procedures, TO SnomedCode)
CREATE UNDIRECTED EDGE CAREPLAN_CODE(FROM Careplans, TO SnomedCode)
CREATE UNDIRECTED EDGE CAREPLAN_REASON_CODE(FROM Careplans, TO SnomedCode)
CREATE UNDIRECTED EDGE CONDITION_CODE(FROM Conditions, TO SnomedCode)
CREATE UNDIRECTED EDGE IMMUNIZATION_CODE(FROM Immunizations, TO SnomedCode)
CREATE UNDIRECTED EDGE OBSERVATION_FOR_PATIENT(FROM Observations, TO Patient)
CREATE UNDIRECTED EDGE OBSERVATION_CODE(FROM Observations, TO SnomedCode)
CREATE UNDIRECTED EDGE ORGANIZATION_ADDRESS(FROM Organizations, TO AddressSynthea)
CREATE UNDIRECTED EDGE IMAGING_CODE(FROM ImagingStudies, TO SnomedCode)
"""
)

'Successfully created edge types: [ALLERGY_CODE].\nSuccessfully created edge types: [DEVICE_CODE].\nSuccessfully created edge types: [PROCEDURE_CODE].\nSuccessfully created edge types: [PROCEDURE_REASON_CODE].\nSuccessfully created edge types: [CAREPLAN_CODE].\nSuccessfully created edge types: [CAREPLAN_REASON_CODE].\nSuccessfully created edge types: [CONDITION_CODE].\nSuccessfully created edge types: [IMMUNIZATION_CODE].\nSuccessfully created edge types: [OBSERVATION_FOR_PATIENT].\nSuccessfully created edge types: [OBSERVATION_CODE].\nSuccessfully created edge types: [ORGANIZATION_ADDRESS].\nSuccessfully created edge types: [IMAGING_CODE].'

## Create Graph

The schema that we just created exists in the **Global** sense. We can have more than one **Graph** per **Solution**, so a **Global** schema allows us to re-use parts or all of that schema across multiple graphs.

We're not going to need to do anything fancy like that for this demo, it's just important to know that any **Graphs** can contain none or any number of elements from the **Global** schema and can even contain elements that are unique to that **Graph's** schema.

In [15]:
conn.gsql(
    """
    CREATE GRAPH MedGraph(Gender, Race, Ethnicity, AddressSynthea, CitySynthea, StateSynthea, CountySynthea, ZipCodeSynthea, 
                 SnomedCode, Patient, Allergies, Device, Medication, Procedures,
                 Careplans, Conditions, Immunizations, Observations, Organizations, Providers, ImagingStudies, Payer, Encounter, 
                 Notes, Symptoms, Attribute, PATIENT_HAS_ATTRIBUTE,
                 PATIENT_HAS_SYMPTOM, PATIENT_NOTE, ADDRESS_CITY_SYNTHEA, ADDRESS_COUNTY_SYNTHEA, ADDRESS_ZIPCODE_SYNTHEA, STATE_HAS_COUNTY_SYNTHEA, 
                 COUNTY_HAS_CITY_SYNTHEA, CITY_HAS_ZIPCODE_SYNTHEA, PATIENT_GENDER,
  PATIENT_ADDRESS, PATIENT_RACE, PATIENT_ETHNICITY, PATIENT_HAS_ALLERGY, ENCOUNTER_FOR_ALLERGY, ALLERGY_CODE,
  PATIENT_HAS_DEVICE, ENCOUNTER_FOR_DEVICE, DEVICE_CODE, PATIENT_HAS_MEDICATION, MEDICATION_PAYER, ENCOUNTER_FOR_MEDICATION,
  MEDICATION_REASON_CODE, MEDICATION_CODE, PROCEDURE_CODE, PROCEDURE_REASON_CODE, PATIENT_HAS_PROCEDURE, ENCOUNTER_FOR_PROCEDURE,
  PATIENT_HAS_CAREPLAN, ENCOUNTER_FOR_CAREPLAN, CAREPLAN_CODE, CAREPLAN_REASON_CODE, PATIENT_HAS_CONDITION, ENCOUNTER_FOR_CONDITION,
  CONDITION_CODE, PATIENT_HAS_IMMUNIZATION, ENCOUNTER_FOR_IMMUNIZATION, IMMUNIZATION_CODE, OBSERVATION_FOR_PATIENT, ENCOUNTER_FOR_OBSERVATION,
  OBSERVATION_CODE, ORGANIZATION_ADDRESS, PROVIDER_HAS_ORGANIZATION, PROVIDER_GENDER, PROVIDER_ADDRESS, PATIENT_HAS_IMAGING, ENCOUNTER_FOR_IMAGING,
  IMAGING_CODE, PAYER_TRANSITION, PAYER_ADDRESS, ENCOUNTER_FOR_PATIENT, ENCOUNTER_UNDER_ORGANIZATION, ENCOUNTER_HAS_PROVIDER, ENCOUNTER_HAS_PAYER,
  ENCOUNTER_CODE, ENCOUNTER_REASON_CODE)
"""
)

'The graph MedGraph is created.'